# COSC2673 - Machine Learning - Assignment 2
### Authors: Louise Platts (S3908818) & Samuel Macintyre (S3888492)

# Classifying Images of Road Traffic Signs

# Introduction

* want to solve two problems 
    - classifying road signs according to sign-shape
    - classifying road signs according to sign-type
* utilises the modified Belgium Traffic Sign Classification Benchmark dataset (as preprepared for this assignment and provided on Canvas)
* problems will be solved via investigations of supervised machine learning algorithms for each of the two problems
* a final independent evaluation will also take place which will involve using the trained models on independently collected data and evaluating performance

#### More about the dataset
* 28x28 gray-scale images 
* possible sign-shapes
    - diamond
    - hex 
    - round 
    - square
    - triangle
* possible sign-types
    - right of way
    - stop
    - bicycle
    - limited traffic
    - no entry
    - no parking
    - roundabout
    - speed
    - traffic directive
    - travel direction
    - continue
    - crossing
    - lane end
    - parking
    - giveway
    - warning


#### CRISP-DM Process 
Our investigation and solution to the problem is guided by the Cross Industry Standard Process for Data Mining (CRISP-DM) process.  Kelleher, Namee and D'Arcy (2015) describe this process as containing key phases which have been modified to suit our problem and will be addressed as follows:
* Data Understanding - examining the data structure, and for our case the file structure as this contains the relevant label for each data point 
* Data Preparation - examining required inputs of machine learning algorithms and models and transforming to suit
* Modelling - apply machine learning algorithms and models to investigate the application and performance of models, and attempt to improve performance via tuning 
* Evaluation - generating performance metrics for models generated, and using this to guide further tuning and improvement of models, finding the ultimate best model type and parameter choice for the problem to solve
* Deployment - using the ultimate model found for each problem and applying it to independent data to implement the model in a real-world situation

#### Libraries Required

* what libraries are available for image classification supervised ml 
* why we made decisions we did - e.g. tensorflow because taught and requested by teacing team ?? [probs not this but lol we have been told] etc. 

In [ ]:
# import statements
# test push

# Data Understanding & Data Preparation

In [ ]:
# read in data

# file directory structure to be used to create label info in df

# Modelling & Evaluation

From rubric:
* baseline model
* cost function
* optimisation

In [ ]:
# baseline + tuning for each 
# at least one for each problem required -> we want to do 2 full models at least

## Ultimate Judgement - Sign-Shape

## Ultimate Judgement - Sign-Type

# Deployment on Independent Data

In [ ]:
# read in and prep of independent data

Can't forget:
* discuss challenges faced in combining new data to model 
    - data transformation into required format per training = 28x28 grey-scale etc.

# References
Kelleher, J., Namee, B. and D'Arcy, A., 2015. Fundamentals of Machine Learning for Predictive Data Analytics. Cambridge: MIT Press.
